# Programming Assignment 2 - Multiple Linear Regression (80 points)

## Implementing Multiple Linear Regression Using Gradient Descent And Stochastic Gradient Descent Algorithm.

In this assignment, you will be implementing Multiple Linear Regression Model on the same data set as in Programming Assignment 1, but here we will be using gradient descent algorithm (GDA) and stochastic gradient descent algorithm (SGDA) to minimize the cost function as taught in the class. The SGDA implementation is optional.

Please add your own print statements to check your code to ensure your code is correct in every step.  (Note: we will not be grading the print statements you add to your code.)

In [2]:
# Run this code cell using Shift + Enter before moving further
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt

# Loading the data set.

In the below code cell, you will load the data using python pandas library as done in the programming_assignment_1b. Use pd.read_csv('File url ', header=None,.... ) with the value of header=None,delim_whitespace=True,names=names,na_values='?' as attributes. The url for the .data file is https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data. This step is same as done in programming_assignment_1b.

In [32]:
# After completing the code in this code cell, run this code cell before moving further.
names =[
    'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 
    'AGE',  'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'PRICE'
]

# TODO - write the pandas command to read the csv file into a dataframe df - 5 points
df= pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data", sep="\s+", names=names)
print(df)


         CRIM    ZN  INDUS  CHAS    NOX     RM    AGE     DIS  RAD    TAX  \
0     0.00632  18.0   2.31     0  0.538  6.575   65.2  4.0900    1  296.0   
1     0.02731   0.0   7.07     0  0.469  6.421   78.9  4.9671    2  242.0   
2     0.02729   0.0   7.07     0  0.469  7.185   61.1  4.9671    2  242.0   
3     0.03237   0.0   2.18     0  0.458  6.998   45.8  6.0622    3  222.0   
4     0.06905   0.0   2.18     0  0.458  7.147   54.2  6.0622    3  222.0   
5     0.02985   0.0   2.18     0  0.458  6.430   58.7  6.0622    3  222.0   
6     0.08829  12.5   7.87     0  0.524  6.012   66.6  5.5605    5  311.0   
7     0.14455  12.5   7.87     0  0.524  6.172   96.1  5.9505    5  311.0   
8     0.21124  12.5   7.87     0  0.524  5.631  100.0  6.0821    5  311.0   
9     0.17004  12.5   7.87     0  0.524  6.004   85.9  6.5921    5  311.0   
10    0.22489  12.5   7.87     0  0.524  6.377   94.3  6.3467    5  311.0   
11    0.11747  12.5   7.87     0  0.524  6.009   82.9  6.2267    5  311.0   

Now, instead of taking just 1 feature i.e 'RM' as we did in Simple Linear Regression, we will use 4 features which are 'LSTAT', 'DIS', 'RM' and 'INDUS'. We choose only these features because they are almost on similar scale. So, there is no need for feature scaling. Fetching the values of 5 columns into a smaller dataframe (say) df1 from df. This step is also same as done in programming_assignment_1b.There we fetched only 2 columns but here we need 5 columns.We need the values in the 'PRICE', 'LSTAT', 'DIS', 'RM' and 'INDUS' columns.

In [52]:
# TODO - Save values from columns 'PRICE', 'LSTAT', 'DIS', 'RM', 'INDUS' into df1 from df - 5 points
df1= df[["PRICE", "LSTAT", "DIS", "RM", "INDUS"]]

# TODO - Remove nan values from df1 and save into df2 - 5 points
df2= df1[~np.isnan(df1)]

# Check the shape of df2. It should be (506,5)
print(df2.shape)
print(df2)


(506, 5)
     PRICE  LSTAT     DIS     RM  INDUS
0     24.0   4.98  4.0900  6.575   2.31
1     21.6   9.14  4.9671  6.421   7.07
2     34.7   4.03  4.9671  7.185   7.07
3     33.4   2.94  6.0622  6.998   2.18
4     36.2   5.33  6.0622  7.147   2.18
5     28.7   5.21  6.0622  6.430   2.18
6     22.9  12.43  5.5605  6.012   7.87
7     27.1  19.15  5.9505  6.172   7.87
8     16.5  29.93  6.0821  5.631   7.87
9     18.9  17.10  6.5921  6.004   7.87
10    15.0  20.45  6.3467  6.377   7.87
11    18.9  13.27  6.2267  6.009   7.87
12    21.7  15.71  5.4509  5.889   7.87
13    20.4   8.26  4.7075  5.949   8.14
14    18.2  10.26  4.4619  6.096   8.14
15    19.9   8.47  4.4986  5.834   8.14
16    23.1   6.58  4.4986  5.935   8.14
17    17.5  14.67  4.2579  5.990   8.14
18    20.2  11.69  3.7965  5.456   8.14
19    18.2  11.28  3.7965  5.727   8.14
20    13.6  21.02  3.7979  5.570   8.14
21    19.6  13.83  4.0123  5.965   8.14
22    15.2  18.72  3.9769  6.142   8.14
23    14.5  19.88  4.0952  5.81

In [76]:
# TODO - Create a vector y having the values of 'PRICE' column and matrix x having the values of
# LSTAT', 'DIS', 'RM' and 'INDUS' columns - 5 points
y = np.array(df2["PRICE"])
x = df2[["LSTAT", "DIS", "RM", "INDUS"]]
x = x.values
# Check the shape of x and y vectors.
print(x.shape)
print(y.shape)

(506, 4)
(506,)


Reshape y to be rank 2. After checking the shape of x and y in the above code cell, we see that x is already rank 2 and y are rank 1 matrix. Before moving ahead, convert y to be rank 2 matrix. For example, I would use the command y=y.reshape(y.shape[0],1) to reshape.

In [77]:
# TODO - Reshape y into a rank 2 matrix - 5 points
y = y.reshape(y.shape[0], 1)
print(y.shape)

(506, 1)


Calculating the value of n i.e. number of training examples. 
Hint: Value of n is equal to the number of rows in either x or y matrix which can be accessed using numpy shape command.

In [78]:
# TODO - Save number of training examples into n and print it - 5 points
n = y.shape[0]
print(n)

506


In [79]:
# TODO - Append a column of ones in x. Use hstack - 5 points

x = np.hstack((np.ones((x.shape[0],1)), x))
print(x)

# Shape of x should be (506,5)
print(x.shape)

[[ 1.      4.98    4.09    6.575   2.31  ]
 [ 1.      9.14    4.9671  6.421   7.07  ]
 [ 1.      4.03    4.9671  7.185   7.07  ]
 ...
 [ 1.      5.64    2.1675  6.976  11.93  ]
 [ 1.      6.48    2.3889  6.794  11.93  ]
 [ 1.      7.88    2.505   6.03   11.93  ]]
(506, 5)


# Cost Function.
Compute the cost: Write the code to compute the cost inside the function. Do not change the function name or function parameters.

In [93]:
def compute_cost(x, y, w, n):
    # Write your code in place of None. Cost can be calculated using a single line of code.
    # Remember w is a vector here.
    
    # TODO - Write the cost function - 10 points
    cost = ( (1/(2*n))*np.sum((y - np.dot(x,w))**2) )
    
    return cost

Before moving ahead, ensure that the code you have written to compute the cost is correct. Just run the below cell.

In [94]:
w_testcase=np.zeros((5,1))
cost_verify= compute_cost(x, y, w_testcase, n)
print(cost_verify)

296.0734584980237


Your output should be equal to 296.073458498. If it's equal to this, then move ahead. Else, re-check your code and re-verify.

# Gradient Descent
Write the code to perform gradient descent in the function below

In [95]:
def gradient_descent(x , y , learning_rate , w , n , num_iters):
    # In place of None, write the updated value of w0 in temp0 and of w1 in temp1
    # TODO - Finish the gradient descent function - 10 points
    
    for i in range(num_iters):
        # derivative vector is given by : X_train.Transpose *  (( X_train * w)- y ) 
        temp =  None
        w = None
       

        if(i%100==0):
            # In place of None, call the cost you just coded above
            cost= None
            print("Cost")
            print(cost)
             
    return w      

Before moving ahead, ensure that your code to update w is correct. Run the code cell below.

In [ ]:
w_testcase=np.zeros((5,1))
g=gradient_descent(x , y , 0.0049 , w_testcase , n , 100000)
print(g[0])

The last output should be: 7.46236374. If it matches your out, then your code is likely to be correct. Otherwise, re-check your code and re-verify.

# Integrating the Batch Gradient Descent Function 

Integrating the above function into a single function multiple_linear_reg_model_gda: This function uses gradient descent algorithm to minimize the cost.

In [ ]:
def multiple_linear_reg_model_gda(x , y , n , learning_rate , num_iters):
    #initialize the values of parameter vector w. It should be a column vector of zeros of dimension(n,1)
    # TODO - Complete the function - 15 points
    w = None
    
    # Calculate the initial cost by calling the function you coded above.
    initial_cost= None
    print("Initial Cost")
    print(initial_cost)
    
    # Calculate the optimized value of gradients by calling the gradient_descent function coded above
    
    w = None
    
    # Calculate the cost with the optimized value of w0 and w1 by calling the cost function.
    
    final_cost = None
    print("Final Cost")
    print(final_cost)
    return w

Now, when you have coded multiple_linear_reg_model_gda function, you can call this function to find the optimized values of parameters w. Before calling the function, set the values of learning_rate and num_iters. You may have to call this function several number of times with different values of num_iters and learning_rate to find the optimal values of w. For some values of learning_rate, it may give an error as the values of cost may reach a very high value(infinity) due to overshooting as discussed in the class. 

In [ ]:
learning_rate = None
num_iters = None

# In place of None, call the multiple_linear_reg_model_gda.
w = None
print(w)

# The value of final cost should be 14.3470049896 or nearly this(depending on the values of learning_rate and num_itersations you choose.)

# Normal Equation Method
Now, we will be writing the code to find the values of parameters w for our multiple linear regression model. This can also be used to cross-check the optimal values of w we just found above using method above. These values should be same (or nearly same).
Instead of writing the code for normal equation in one line, you can break this into 3 parts: First calculate q=inverse of (dot of (X.T,X)) (these are pseudo commands, use original numpy commands to calculate q). Then z= dot of ( X.T , y) and then w_vec= dot of (q,z). Here, w_vec is vector of dimension (5,1) having two values. Example w0=w_vec[0][0].

Note : Do not append a column of ones in x because you have already done before. 

In [ ]:
# Write the code below
# TODO - Write the normal equation method - 15 points

q = None
z = None
w_vec = None


In [ ]:
print(w_vec)

The values of w you just got above should be approximately same as the ones you got using multiple_linear_reg_model_gda.
This assignment ends here.